In [1]:
import pandas as pd
import numpy as np
from htm.bindings.sdr import SDR
from htm.algorithms import SpatialPooler, Classifier
from htm.algorithms import TemporalMemory as TM

In [2]:
day2 = pd.read_csv('D:/2021\하계코옵/업무폴더/16_Dataset/archive/PRONTO benchmark case study/Pre-processed data/Aligned and labelled alarm and process data/Testday2_merged.csv')
day3 = pd.read_csv('D:/2021\하계코옵/업무폴더/16_Dataset/archive/PRONTO benchmark case study/Pre-processed data/Aligned and labelled alarm and process data/Testday3_merged.csv')
day4 = pd.read_csv('D:/2021\하계코옵/업무폴더/16_Dataset/archive/PRONTO benchmark case study/Pre-processed data/Aligned and labelled alarm and process data./Testday4_merged.csv')
data_arr = pd.concat([day2, day3, day4], axis = 0, ignore_index=True, join = 'inner')

In [3]:
data_arr.shape

(45420, 29)

In [4]:
data_arr.columns

Index(['FIC301', 'FIC302', 'LI405', 'PT312', 'VC303', 'VC401', 'VC403',
       'VC404', 'VC405', 'VC406', 'VLV303', 'AirIn', 'Air.T', 'Air.P',
       'WaterIn', 'Water.T', 'Water.Density', 'Mixture.zone.P',
       'riser.outlet.P', 'P.topsep', 'FR.topsep.gas', 'FR.topsep.liquid',
       'P_3phase', 'Air.Valve', 'Water.level', 'Water.coalescer',
       'Water.level.valve', 'water.tank.level', 'Fault'],
      dtype='object')

## failure

In [5]:
input_arr1 = data_arr.loc[:, 'FIC301':'water.tank.level']
target_arr1 = data_arr.loc[:, 'Fault']

In [6]:
input_arr1.shape

(45420, 17)

In [7]:
target_arr1[target_arr1 == 'Normal'] = 0
target_arr1[target_arr1 == 'Air blockage'] = 1
target_arr1[target_arr1 == 'Air leakage'] = 2
target_arr1[target_arr1 == 'Diverted flow'] = 3
target_arr1[target_arr1 == 'Slugging'] = 4

C:\Users\user\anaconda3\envs\htm\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\anaconda3\envs\htm\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\anaconda3\envs\htm\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from th

In [8]:
input_data1 = input_arr1.to_numpy()
target_data1 = target_arr1.to_numpy(dtype = np.int)

C:\Users\user\anaconda3\envs\htm\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [9]:
test_index = int(input_data1.shape[0] * 0.8)

In [10]:
input_data1 = input_data1.reshape(-1, 28, 1)

In [11]:
train_input1 = input_data1[:test_index, :]
test_input1 = input_data1[test_index:,:] 
train_target1 = target_data1[:test_index]
test_target1 = target_data1[test_index: ]

print(train_input1.shape, test_input1.shape, train_target1.shape, test_target1.shape)

(36336, 17, 1) (9084, 17, 1) (36336,) (9084,)


In [12]:
parameters = {
    'potentialRadius': 3,
    'boostStrength': 7.0,
    'columnDimensions': [50, 50],
    'dutyCyclePeriod': 1402,
    'localAreaDensity': 0.1,
    'minPctOverlapDutyCycle': 0.2,
    'potentialPct': 0.1,
    'stimulusThreshold': 6,
    'synPermActiveInc': 0.14,
    'synPermConnected': 0.5,
    'synPermInactiveDec': 0.02
}

In [13]:
def encode(data, out):
    out.dense = data >= np.mean(data)


In [14]:
enc = SDR([28, 1])
sp = SpatialPooler(
    inputDimensions            = enc.dimensions,
    columnDimensions           = parameters['columnDimensions'],
    potentialRadius            = parameters['potentialRadius'],
    potentialPct               = parameters['potentialPct'],
    globalInhibition           = True,
    localAreaDensity           = parameters['localAreaDensity'],
    stimulusThreshold          = int(round(parameters['stimulusThreshold'])),
    synPermInactiveDec         = parameters['synPermInactiveDec'],
    synPermActiveInc           = parameters['synPermActiveInc'],
    synPermConnected           = parameters['synPermConnected'],
    minPctOverlapDutyCycle     = parameters['minPctOverlapDutyCycle'],
    dutyCyclePeriod            = int(round(parameters['dutyCyclePeriod'])),
    boostStrength              = parameters['boostStrength'],
    seed                       = 0, # this is important, 0="random" seed which changes on each invocation
    spVerbosity                = 99,
    wrapAround                 = False
)
columns = SDR(sp.getColumnDimensions())
sdrc = Classifier()

------------CPP SpatialPooler Parameters ------------------
iterationNum                = 0
iterationLearnNum           = 0
numInputs                   = 17
numColumns                  = 2500
numActiveColumnsPerInhArea  = 0
potentialPct                = 0.1
globalInhibition            = 1
localAreaDensity            = 0.1
stimulusThreshold           = 6
synPermActiveInc            = 0.14
synPermInactiveDec          = 0.02
synPermConnected            = 0.5
minPctOverlapDutyCycles     = 0.2
dutyCyclePeriod             = 1402
boostStrength               = 7
spVerbosity                 = 99
wrapAround                  = 0
version                     = 2
CPP SP seed                 = 0


In [15]:
for i in range(len(train_input1)):
    #enc.dense = train_input1[i]
    encode(train_input1[i], enc)
    sp.compute(enc, True, columns)
    sdrc.learn(columns, train_target1[i])

In [16]:
print(enc.dense, train_input1[0])

[[1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]] [[1.21053596e+02]
 [1.56816597e+01]
 [1.30110705e+00]
 [9.63085290e-02]
 [2.37480793e+01]
 [9.97596497e+02]
 [1.24009705e+00]
 [1.02590704e+00]
 [1.02717495e+00]
 [4.90892000e-04]
 [6.48449060e-02]
 [1.00591600e+00]
 [2.50923100e+01]
 [6.22431107e+01]
 [9.86426773e+01]
 [2.25735593e+00]
 [1.44312096e+00]]


In [17]:
print(str(sp))

Spatial Pooler Connections:
    Inputs (17) ~> Outputs (2500) via Segments (2500)
    Segments on Cell Min/Mean/Max 1 / 1 / 1
    Potential Synapses on Segment Min/Mean/Max 0 / 0.88 / 1
    Connected Synapses on Segment Min/Mean/Max 0 / 0.88 / 1
    Synapses Dead (0%) Saturated (0.852727%)
    Synapses pruned (0%) Segments pruned (0%)
    Buffer for destroyed synapses: 0    Buffer for destroyed segments: 0



In [18]:
score = 0
for i in range(len(train_input1)):
    encode(train_input1[i], enc)
    sp.compute(enc, False, columns)
    if(train_target1[i] == np.argmax(sdrc.infer(columns))):
        score += 1
score = score / len(train_input1)

In [19]:
print(score)

0.16785006605019814


In [20]:
score = 0
for i in range(len(test_input1)):
    encode(train_input1[i], enc)
    sp.compute(enc, False, columns)
    if(test_target1[i] == np.argmax(sdrc.infer(columns))):
        score += 1
score = score / len(test_input1)

In [21]:
print(score)

0.6384852487890798


In [22]:
enc = SDR(train_input1.shape[0] * train_input1.shape[1])

In [23]:
for i in range(train_input1.shape[0]):
    for j in range(train_input1.shape[1]):
        enc.dense = train_input1[i, j]

RuntimeError: Exception: py_SDR.cpp(157) message: CHECK FAILED: "(UInt) buf.shape[0] == self.size" Bad input array size! expected 617712, got 1

In [25]:
train_flatten = train_input1.reshape(-1)
print(train_flatten.shape)

(617712,)


In [26]:
enc.dense = train_flatten

In [34]:
enc = SDR([17, 1])

In [35]:
tm = TM(columnDimensions = (enc.size,),
        cellsPerColumn=5,       # default: 32
        minThreshold= 1,         # default: 10
        activationThreshold=1,  # default: 13
        initialPermanence=0.5,  # default: 0.21
        )

In [36]:
train_input1.shape

(36336, 17, 1)

In [37]:
for i in range(len(train_input1)):
    enc.dense = train_input1[i]
    tm.compute(enc, True)
    

RuntimeError: Exception: Sdr.cpp(368) message: CHECK FAILED: "inputs[i]->dimensions == dimensions" 

In [ ]:
sdr = tm.getActiveCells()

print(sdr.dense.flatten())
print(np.sum(sdr.dense.flatten()))
print(np.min(sdr.dense.flatten()))
print(len(sdr.dense.flatten()))


[1 1 1 ... 1 1 1]
2438980
0
3088560


In [ ]:
score = 0
for i in range(100):
    if(train_flatten[i] == sdr.dense.flatten()[i*5]):
        score += 1
score = score / len(train_flatten)
print(score)

0.0


## alarm

In [ ]:
target_arr2 = data_arr.loc[:,'FIC301':'VLV303']
target_arr2 = target_arr2.to_numpy(dtype = np.int)
target_int = []
one_hot = 0
for x in target_arr2:
    for i in range(len(target_arr2[0])):
        # failure situation
        if(x[i] == 1):
            target_int.append(i + 1)
            one_hot = 1
            break # priority encoder
    # normal situation
    if(one_hot == 0):
        target_int.append(0)
    one_hot = 0
        
target_int = np.array(target_int, dtype = np.int)

C:\Users\user\anaconda3\envs\htm\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
C:\Users\user\anaconda3\envs\htm\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additi

In [ ]:
input_arr2 = data_arr.loc[:, 'AirIn':'water.tank.level']
input_arr2 = input_arr2.to_numpy(dtype = np.float32)

In [ ]:
train_input2 = input_arr2[:test_index, :]
test_input2 = input_arr2[test_index: , :]
train_target2 = target_int[:test_index]
test_target2 = target_int[test_index:]
print(train_input2.shape, test_input2.shape, train_target2.shape, test_target2.shape)


(36336, 17) (9084, 17) (36336,) (9084,)


In [ ]:
enc = SDR([17, 1])